In [1]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]


train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

In [4]:
voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [5]:
glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

In [6]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [7]:
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [8]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         1373200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         16512 

In [9]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)


In [10]:
model.compile(optimizer=SGD(learning_rate=0.01),loss='mean_absolute_error',metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [11]:
history = model.fit(x=x_train,y=y_train,batch_size=5,epochs=100)

Epoch 1/100
1125/1125 [==============================] - 10s 8ms/step - loss: 0.8900 - mean_squared_error: 1.2850 - mean_absolute_error: 0.8900 - mean_absolute_percentage_error: 23.9710
Epoch 2/100
1125/1125 [==============================] - 9s 8ms/step - loss: 0.7369 - mean_squared_error: 0.8488 - mean_absolute_error: 0.7369 - mean_absolute_percentage_error: 20.2422
Epoch 3/100
1125/1125 [==============================] - 9s 8ms/step - loss: 0.7044 - mean_squared_error: 0.7751 - mean_absolute_error: 0.7044 - mean_absolute_percentage_error: 19.5005
Epoch 4/100
1125/1125 [==============================] - 9s 8ms/step - loss: 0.6675 - mean_squared_error: 0.7011 - mean_absolute_error: 0.6675 - mean_absolute_percentage_error: 18.4719
Epoch 5/100
1125/1125 [==============================] - 9s 8ms/step - loss: 0.6312 - mean_squared_error: 0.6288 - mean_absolute_error: 0.6312 - mean_absolute_percentage_error: 17.7459
Epoch 6/100
1125/1125 [==============================] - 9s 8ms/step - los

In [12]:
mse = history.history['mean_squared_error']
mae = history.history['mean_absolute_error']
mpe = history.history['mean_absolute_percentage_error']

scores = pd.DataFrame()
scores["MSE"] = mse
scores["MAE"] = mae
scores["% Error"] = mpe

In [15]:
# With ADAM
scores = model.evaluate(x=x_val,y=y_val)
scores[1:]


model.predict(x_val)

44/44 [==============================] - 0s 9ms/step - loss: 0.6653 - mean_squared_error: 0.7157 - mean_absolute_error: 0.6653 - mean_absolute_percentage_error: 18.3475


array([[4.41891  ],
       [4.1580715],
       [4.4683447],
       ...,
       [4.6913967],
       [2.1770558],
       [5.2930727]], dtype=float32)